In [1]:
import requests
import os

# 테스트 설정
BASE_URL = "http://localhost:8000/api" 
ROLES = {
    "DIRECTOR": "token_for_director",
    "MANAGER": "token_for_manager",
    "STAFF": "token_for_staff"
}

ENDPOINTS = [
    {"path": "/detect/", "allowed": ["DIRECTOR", "MANAGER", "STAFF"]},
    {"path": "/admin/stats/", "allowed": ["DIRECTOR"]},
    {"path": "/history/delete/", "allowed": ["DIRECTOR", "MANAGER"]}
]


In [2]:
# 1. 테스트 실행 및 결과 수집
results = []
print("=== [QA] 역할별 권한 분리 테스트 시작 ===")
for endpoint in ENDPOINTS:
    path = endpoint['path']
    print(f"\n경로: {path}")
    for role, token in ROLES.items():
        headers = {"Authorization": f"Bearer {token}"}
        try:
            response = requests.get(f"{BASE_URL}{path}", headers=headers)
            status_code = response.status_code
        except Exception as e:
            status_code = f"Error({e})"
        
        is_allowed = role in endpoint['allowed']
        status_match = (str(status_code) != "403") == is_allowed
        result_status = "PASS" if status_match else "FAIL"
        
        res_text = f" - [{role}] 접근 시도 -> 코드: {status_code} | 결과: {result_status}"
        print(res_text)
        results.append({"path": path, "role": role, "text": res_text})
        

=== [QA] 역할별 권한 분리 테스트 시작 ===

경로: /detect/
 - [DIRECTOR] 접근 시도 -> 코드: Error(HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/detect/ (Caused by NewConnectionError("HTTPConnection(host='localhost', port=8000): Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다"))) | 결과: PASS
 - [MANAGER] 접근 시도 -> 코드: Error(HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/detect/ (Caused by NewConnectionError("HTTPConnection(host='localhost', port=8000): Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다"))) | 결과: PASS
 - [STAFF] 접근 시도 -> 코드: Error(HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/detect/ (Caused by NewConnectionError("HTTPConnection(host='localhost', port=8000): Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다"))) | 결과: PASS

경로: /admin/stats/
 - [DIRECTOR] 접근 시도 -> 코드: Error

In [3]:
# 2. 결과 저장 로직
save_path = r"..\qa\model_qa_results"
if not os.path.exists(save_path):
    os.makedirs(save_path)

file_name = os.path.join(save_path, "auth_test_report.txt")

with open(file_name, 'w', encoding='utf-8') as f:
    f.write("=== [QA] 역할별 권한 분리 테스트 리포트 ===\n")
    current_path = ""
    for item in results:
        if current_path != item['path']:
            current_path = item['path']
            f.write(f"\n경로: {current_path}\n")
        f.write(item['text'] + "\n")
    f.write("\n=== 테스트 완료 ===\n")

print(f"\n정밀 성능 평가 결과가 저장되었습니다: {file_name}")



정밀 성능 평가 결과가 저장되었습니다: D:\final_project\qa\model_qa_results\auth_test_report.txt
